In [1]:
import os
import itertools
import networkx as nx
import matplotlib as plt

import constants
from run_experiment import run_experiment
from run_experiment import get_run_graph
from coup_matchup_environment import CoupMatchupEnvironment

In [102]:
# Run experiment if data file is not present
experiment_result_path = "../data/results.txt"
if not os.path.isfile(experiment_result_path):
    run_experiment(path=experiment_result_path, verbose=False, num_cores=1)

card_pairs = list(itertools.product(constants.CARDS, constants.CARDS))
matchups = list(itertools.product(card_pairs, card_pairs))

# matchup_results[(p1_card_1,p1_card_2)][(p2_card_1,p2_card_2)] = winner
matchup_results = dict(zip([card_pair for card_pair in card_pairs], [dict(zip([card_pair for card_pair in card_pairs], [None for _ in card_pairs])) for _ in card_pairs]))
with open(experiment_result_path, "r") as file:
    for line in file:
        matchup_data = line.split(",")
        # Remove () and ' from the strings
        for i, data in enumerate(matchup_data):
            matchup_data[i] = data.replace("(", "").replace("\'", "").replace(")","").strip()
        # Add all orientations of the matchup since run_experiment does not calculate the winner for equivalent matchups (ie duke,assassin vs captain,duke == assassin,duke vs duke,captain)
        matchup_results[(matchup_data[0], matchup_data[1])][(matchup_data[2], matchup_data[3])] = matchup_data[4]
        matchup_results[(matchup_data[1], matchup_data[0])][(matchup_data[2], matchup_data[3])] = matchup_data[4]
        matchup_results[(matchup_data[0], matchup_data[1])][(matchup_data[3], matchup_data[2])] = matchup_data[4]
        matchup_results[(matchup_data[1], matchup_data[0])][(matchup_data[3], matchup_data[2])] = matchup_data[4]

for matchup in matchups:
    assert matchup_results[matchup[0]][matchup[1]] is not None, f"Error no result for matchup {matchup[0]} vs {matchup[1]}"

In [103]:
print(matchup_results[(constants.CAPTAIN, constants.DUKE)][(constants.DUKE, constants.ASSASSIN)])

2


In [ ]:
p1_cards = (constants.DUKE, constants.DUKE)
p2_cards = (constants.CAPTAIN, constants.CAPTAIN)
matchup = CoupMatchupEnvironment(p1_cards, p2_cards)
matchup.solve(verbose=False)
pi1 = matchup.get_policy(1)
pi2 = matchup.get_policy(2)
graph = get_run_graph(matchup, pi1, pi2)

with open('../data/graph.csv', 'w') as file:
    file.write(f"source,target\n")
    for source in graph.keys():
        for target in graph[source]:
            file.write(f"{str(source).replace(',', '.')},{str(target).replace(',', '.')}\n")